# Panduan Lengkap: Pipeline Data Cuaca (Copernicus ke Android MVVM)

Panduan ini akan memandu Anda melalui seluruh proses end-to-end, dimulai dari mendapatkan kredensial API Copernicus, mengunduh dan memproses data 10 tahun di Google Colab, hingga mengintegrasikan database final ke dalam aplikasi Android modern yang menggunakan Jetpack Compose dan arsitektur MVVM.

## Fase 1: Konfigurasi Lingkungan Google Colab

Langkah pertama adalah menyiapkan Google Colab untuk berkomunikasi dengan server Copernicus.

### Langkah 1.1: Dapatkan Kredensial API Copernicus Anda

Setelah Anda login ke portal Copernicus Climate Data Store (CDS):
- Klik nama Anda di pojok kanan atas.
- Navigasi ke halaman profil Anda.
- Temukan bagian "API key".
- Anda akan melihat sebuah kotak yang menampilkan url dan key Anda. Salin nilai key (ini adalah satu token akses personal yang panjang).
- Biarkan halaman ini tetap terbuka. Anda akan segera menempelkan nilai ini.

### Langkah 1.2: Instal Pustaka cdsapi di Colab

Di sel (cell) baru Google Colab, jalankan perintah berikut untuk menginstal dan meningkatkan (upgrade) pustaka Python ke versi terbaru, seperti yang disarankan oleh pesan error Anda:

In [1]:
!pip install cdsapi==0.7.7 xarray pandas tqdm

### Langkah 1.3: Konfigurasikan Kredensial API Anda di Colab

Jalankan kode Python berikut di sel Colab baru. Ganti 'GANTI_DENGAN_TOKEN_ANDA_DARI_WEBSITE' dengan nilai key (Personal Access Token) Anda. Perhatikan bahwa url telah diperbaiki (tanpa /v2).

In [2]:
import os

# GANTI DENGAN KREDENSIAL ANDA
YOUR_API_TOKEN = "25171a66-2fb9-4d0e-95ff-ecfd24c9d6d4" 

# Kode ini akan membuat file konfigurasi .cdsapirc di lingkungan Colab
# URL telah diperbaiki ke 'https://cds.climate.copernicus.eu/api'
api_key_content = f"""
url: https://cds.climate.copernicus.eu/api
key: {YOUR_API_TOKEN}
"""

with open(f"{os.path.expanduser('~')}/.cdsapirc", "w") as f:
    f.write(api_key_content)

print("File .cdsapirc berhasil dibuat/diperbarui!")

File .cdsapirc berhasil dibuat/diperbarui!


## Fase 2: Akuisisi Data Massal (Mengunduh Data 5 Tahun)



### Langkah 2.1: Tentukan Parameter Permintaan Anda

Parameter tetap sama, tetapi kita akan menerapkannya dalam loop per-bulan.
- Dataset: 'reanalysis-era5-single-levels'
- Variabel: 2m_temperature, total_precipitation, sea_surface_temperature, significant_height_of_combined_wind_waves_and_swell
- Area (Aceh): [6.0, 94.5, 5.0, 96.0]
- Format: netcdf

### Langkah 2.2: Skrip Python untuk Pengambilan Data (Loop per Bulan)

Ganti skrip lama Anda dengan yang ini. Ini adalah skrip yang diperbaiki yang mengunduh data satu bulan pada satu waktu untuk menghindari error "Request is too large".

In [ ]:
# Script untuk tahun 2019
import cdsapi
import time
import os
from tqdm import tqdm

# Buat folder data_iklim jika belum ada
os.makedirs('data_iklim', exist_ok=True)

client = cdsapi.Client()

year = '2019'
all_months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
all_days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
all_times = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']

successful_downloads = 0
failed_downloads = 0

print(f"Memulai download untuk tahun {year}...")

for month in tqdm(all_months, desc=f"Bulan {year}"):
    target_file = f'data_iklim/aceh_data_{year}_{month}.nc'
    print(f"\nMengirimkan permintaan untuk {year}-{month}...")
    
    max_retries = 3
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            client.retrieve(
                'reanalysis-era5-single-levels',
                {
                    'product_type': 'reanalysis',
                    'variable': [
                        '2m_temperature', 'total_precipitation',
                        'sea_surface_temperature', 'significant_height_of_combined_wind_waves_and_swell'
                    ],
                    'year': year,
                    'month': month,
                    'day': all_days,
                    'time': all_times,
                    'area': [6.0, 94.5, 5.0, 96.0],
                    'format': 'netcdf',
                },
                target_file
            )
            print(f"--- BERHASIL: {target_file} ---")
            successful_downloads += 1
            break
        except Exception as e:
            retry_count += 1
            print(f"!!! GAGAL {retry_count}/{max_retries}: {e}")
            if retry_count < max_retries:
                time.sleep(60 * retry_count)
            else:
                failed_downloads += 1

print(f"\nSelesai untuk {year}: Berhasil {successful_downloads}, Gagal {failed_downloads}")

Memulai download untuk tahun 2019...


Bulan 2019:   0%|          | 0/12 [00:00<?, ?it/s]


Mengirimkan permintaan untuk 2019-01...


2025-11-25 01:20:10,947 INFO Request ID is 2b99ffa2-9812-4fe3-84cc-70f2e0b4d8e4
INFO:ecmwf.datastores.legacy_client:Request ID is 2b99ffa2-9812-4fe3-84cc-70f2e0b4d8e4
INFO:ecmwf.datastores.legacy_client:Request ID is 2b99ffa2-9812-4fe3-84cc-70f2e0b4d8e4
2025-11-25 01:20:11,084 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-11-25 01:20:11,084 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-11-25 01:20:20,331 INFO status has been updated to running
2025-11-25 01:20:20,331 INFO status has been updated to running
INFO:ecmwf.datastores.legacy_client:status has been updated to running
INFO:ecmwf.datastores.legacy_client:status has been updated to running
2025-11-25 01:24:32,932 INFO status has been updated to successful
2025-11-25 01:24:32,932 INFO status has been updated to successful
INFO:ecmwf.datastores.legacy_client:status has been updated to succe

e4e837efa11fdac314baf7d90b671816.zip:   0%|          | 0.00/340k [00:00<?, ?B/s]

...

59efd2d674997b47ae216675c61ac82b.zip:   0%|          | 0.00/322k [00:00<?, ?B/s]

Bulan 2019:  17%|█▋        | 2/12 [08:48<44:03, 264.33s/it]

--- BERHASIL: aceh_data_2019_02.nc ---

Mengirimkan permintaan untuk 2019-03...


2025-11-25 01:28:59,446 INFO Request ID is fb7d2f1e-3577-47d5-b0bd-c1cfe2c03657
INFO:ecmwf.datastores.legacy_client:Request ID is fb7d2f1e-3577-47d5-b0bd-c1cfe2c03657
INFO:ecmwf.datastores.legacy_client:Request ID is fb7d2f1e-3577-47d5-b0bd-c1cfe2c03657
2025-11-25 01:28:59,795 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-11-25 01:28:59,795 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-11-25 01:29:05,373 INFO status has been updated to running
INFO:ecmwf.datastores.legacy_client:status has been updated to running
2025-11-25 01:29:05,373 INFO status has been updated to running
INFO:ecmwf.datastores.legacy_client:status has been updated to running
Bulan 2019:  17%|█▋        | 2/12 [15:38<1:18:13, 469.31s/it]



KeyboardInterrupt: 

In [ ]:
# Script untuk tahun 2020
import cdsapi
import time
import os
from tqdm import tqdm

# Buat folder data_iklim jika belum ada
os.makedirs('data_iklim', exist_ok=True)

client = cdsapi.Client()

year = '2020'
all_months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
all_days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
all_times = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']

successful_downloads = 0
failed_downloads = 0

print(f"Memulai download untuk tahun {year}...")

for month in tqdm(all_months, desc=f"Bulan {year}"):
    target_file = f'data_iklim/aceh_data_{year}_{month}.nc'
    print(f"\nMengirimkan permintaan untuk {year}-{month}...")
    
    max_retries = 3
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            client.retrieve(
                'reanalysis-era5-single-levels',
                {
                    'product_type': 'reanalysis',
                    'variable': [
                        '2m_temperature', 'total_precipitation',
                        'sea_surface_temperature', 'significant_height_of_combined_wind_waves_and_swell'
                    ],
                    'year': year,
                    'month': month,
                    'day': all_days,
                    'time': all_times,
                    'area': [6.0, 94.5, 5.0, 96.0],
                    'format': 'netcdf',
                },
                target_file
            )
            print(f"--- BERHASIL: {target_file} ---")
            successful_downloads += 1
            break
        except Exception as e:
            retry_count += 1
            print(f"!!! GAGAL {retry_count}/{max_retries}: {e}")
            if retry_count < max_retries:
                time.sleep(60 * retry_count)
            else:
                failed_downloads += 1

print(f"\nSelesai untuk {year}: Berhasil {successful_downloads}, Gagal {failed_downloads}")

In [ ]:
# Script untuk tahun 2021
import cdsapi
import time
import os
from tqdm import tqdm

# Buat folder data_iklim jika belum ada
os.makedirs('data_iklim', exist_ok=True)

client = cdsapi.Client()

year = '2021'
all_months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
all_days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
all_times = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']

successful_downloads = 0
failed_downloads = 0

print(f"Memulai download untuk tahun {year}...")

for month in tqdm(all_months, desc=f"Bulan {year}"):
    target_file = f'data_iklim/aceh_data_{year}_{month}.nc'
    print(f"\nMengirimkan permintaan untuk {year}-{month}...")
    
    max_retries = 3
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            client.retrieve(
                'reanalysis-era5-single-levels',
                {
                    'product_type': 'reanalysis',
                    'variable': [
                        '2m_temperature', 'total_precipitation',
                        'sea_surface_temperature', 'significant_height_of_combined_wind_waves_and_swell'
                    ],
                    'year': year,
                    'month': month,
                    'day': all_days,
                    'time': all_times,
                    'area': [6.0, 94.5, 5.0, 96.0],
                    'format': 'netcdf',
                },
                target_file
            )
            print(f"--- BERHASIL: {target_file} ---")
            successful_downloads += 1
            break
        except Exception as e:
            retry_count += 1
            print(f"!!! GAGAL {retry_count}/{max_retries}: {e}")
            if retry_count < max_retries:
                time.sleep(60 * retry_count)
            else:
                failed_downloads += 1

print(f"\nSelesai untuk {year}: Berhasil {successful_downloads}, Gagal {failed_downloads}")

In [ ]:
# Script untuk tahun 2022
import cdsapi
import time
import os
from tqdm import tqdm

# Buat folder data_iklim jika belum ada
os.makedirs('data_iklim', exist_ok=True)

client = cdsapi.Client()

year = '2022'
all_months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
all_days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
all_times = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']

successful_downloads = 0
failed_downloads = 0

print(f"Memulai download untuk tahun {year}...")

for month in tqdm(all_months, desc=f"Bulan {year}"):
    target_file = f'data_iklim/aceh_data_{year}_{month}.nc'
    print(f"\nMengirimkan permintaan untuk {year}-{month}...")
    
    max_retries = 3
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            client.retrieve(
                'reanalysis-era5-single-levels',
                {
                    'product_type': 'reanalysis',
                    'variable': [
                        '2m_temperature', 'total_precipitation',
                        'sea_surface_temperature', 'significant_height_of_combined_wind_waves_and_swell'
                    ],
                    'year': year,
                    'month': month,
                    'day': all_days,
                    'time': all_times,
                    'area': [6.0, 94.5, 5.0, 96.0],
                    'format': 'netcdf',
                },
                target_file
            )
            print(f"--- BERHASIL: {target_file} ---")
            successful_downloads += 1
            break
        except Exception as e:
            retry_count += 1
            print(f"!!! GAGAL {retry_count}/{max_retries}: {e}")
            if retry_count < max_retries:
                time.sleep(60 * retry_count)
            else:
                failed_downloads += 1

print(f"\nSelesai untuk {year}: Berhasil {successful_downloads}, Gagal {failed_downloads}")

In [ ]:
# Script untuk tahun 2023
import cdsapi
import time
import os
from tqdm import tqdm

# Buat folder data_iklim jika belum ada
os.makedirs('data_iklim', exist_ok=True)

client = cdsapi.Client()

year = '2023'
all_months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
all_days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
all_times = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']

successful_downloads = 0
failed_downloads = 0

print(f"Memulai download untuk tahun {year}...")

for month in tqdm(all_months, desc=f"Bulan {year}"):
    target_file = f'data_iklim/aceh_data_{year}_{month}.nc'
    print(f"\nMengirimkan permintaan untuk {year}-{month}...")
    
    max_retries = 3
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            client.retrieve(
                'reanalysis-era5-single-levels',
                {
                    'product_type': 'reanalysis',
                    'variable': [
                        '2m_temperature', 'total_precipitation',
                        'sea_surface_temperature', 'significant_height_of_combined_wind_waves_and_swell'
                    ],
                    'year': year,
                    'month': month,
                    'day': all_days,
                    'time': all_times,
                    'area': [6.0, 94.5, 5.0, 96.0],
                    'format': 'netcdf',
                },
                target_file
            )
            print(f"--- BERHASIL: {target_file} ---")
            successful_downloads += 1
            break
        except Exception as e:
            retry_count += 1
            print(f"!!! GAGAL {retry_count}/{max_retries}: {e}")
            if retry_count < max_retries:
                time.sleep(60 * retry_count)
            else:
                failed_downloads += 1

print(f"\nSelesai untuk {year}: Berhasil {successful_downloads}, Gagal {failed_downloads}")

In [ ]:
# Script untuk tahun 2024
import cdsapi
import time
import os
from tqdm import tqdm

# Buat folder data_iklim jika belum ada
os.makedirs('data_iklim', exist_ok=True)

client = cdsapi.Client()

year = '2024'
all_months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
all_days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
all_times = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']

successful_downloads = 0
failed_downloads = 0

print(f"Memulai download untuk tahun {year}...")

for month in tqdm(all_months, desc=f"Bulan {year}"):
    target_file = f'data_iklim/aceh_data_{year}_{month}.nc'
    print(f"\nMengirimkan permintaan untuk {year}-{month}...")
    
    max_retries = 3
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            client.retrieve(
                'reanalysis-era5-single-levels',
                {
                    'product_type': 'reanalysis',
                    'variable': [
                        '2m_temperature', 'total_precipitation',
                        'sea_surface_temperature', 'significant_height_of_combined_wind_waves_and_swell'
                    ],
                    'year': year,
                    'month': month,
                    'day': all_days,
                    'time': all_times,
                    'area': [6.0, 94.5, 5.0, 96.0],
                    'format': 'netcdf',
                },
                target_file
            )
            print(f"--- BERHASIL: {target_file} ---")
            successful_downloads += 1
            break
        except Exception as e:
            retry_count += 1
            print(f"!!! GAGAL {retry_count}/{max_retries}: {e}")
            if retry_count < max_retries:
                time.sleep(60 * retry_count)
            else:
                failed_downloads += 1

print(f"\nSelesai untuk {year}: Berhasil {successful_downloads}, Gagal {failed_downloads}")

### 2.3 Instruksi untuk Running Berkala

Langkah 2.2 telah dipisahkan menjadi cell-cell terpisah untuk setiap tahun (2019-2024) agar memungkinkan download data secara bertahap dan berkala. Ini membantu menghindari overload pada CDS API dan memungkinkan Anda menjalankan download per tahun sesuai kebutuhan.

**Cara menggunakan:**
- Jalankan cell konfigurasi API terlebih dahulu (langkah 2.1).
- Pilih tahun yang ingin Anda download, lalu jalankan cell Python yang sesuai untuk tahun tersebut.
- Setiap cell akan mendownload data untuk 12 bulan dalam satu tahun, dengan retry logic dan progress tracking.
- Jika download gagal untuk bulan tertentu, script akan mencoba ulang hingga 3 kali dengan delay yang meningkat.
- Anda dapat menjalankan beberapa tahun secara berturut-turut atau menunggu antar tahun untuk menghindari batasan API.

**Catatan penting:**
- Pastikan token CDS API Anda valid dan memiliki kuota yang cukup.
- Download untuk satu tahun penuh dapat memakan waktu beberapa jam tergantung pada antrian CDS.
- Jika Anda hanya perlu data untuk bulan tertentu, edit script untuk menjalankan loop bulan yang diinginkan saja.

## Fase 3: Pemrosesan dan Transformasi Data

Setelah data berhasil diunduh per tahun, langkah berikutnya adalah memproses dan mentransformasi data NetCDF menjadi format yang siap untuk database Android.

### Langkah 3.1: Instal Pustaka Pemrosesan

Jalankan di sel baru:

In [ ]:
!pip install xarray pandas

### Langkah 3.1: Muat dan Gabungkan Semua File .nc

Kode berikut akan memuat semua file aceh_data_*.nc yang telah diunduh dari berbagai tahun, menggabungkannya menjadi satu set data besar, dan mengubahnya menjadi tabel Pandas (DataFrame). Pastikan Anda telah menjalankan download untuk tahun-tahun yang diinginkan sebelum menjalankan langkah ini.

In [ ]:
import xarray as xr
import pandas as pd

# Muat dan gabungkan SEMUA file.nc yang telah diunduh
print("Memuat dan menggabungkan file NetCDF...")
# Tanda bintang (*) di 'data_iklim/aceh_data_*.nc' akan menemukan semua 132+ file
# (data_iklim/aceh_data_2014_01.nc, data_iklim/aceh_data_2014_02.nc, dst.)
ds = xr.open_mfdataset('data_iklim/aceh_data_*.nc')

# Konversi ke DataFrame Pandas
print("Mengonversi ke DataFrame Pandas...")
df_hourly = ds.to_dataframe()

# Data memiliki multi-indeks (lat, lon, time). Kita ratakan dan ambil rata-rata
# dari titik grid (jika ada beberapa)
df_hourly = df_hourly.reset_index()
df_hourly = df_hourly.groupby('time').mean(numeric_only=True)

# Ganti nama kolom agar lebih mudah dibaca
df_hourly = df_hourly.rename(columns={
    't2m': 'temp_celsius',
    'tp': 'precipitation_mm',
    'sst': 'sea_surface_temp_c',
    'shww': 'wave_height_meters'
})

# Konversi suhu dari Kelvin ke Celsius (jika perlu, t2m & sst sudah dalam K)
df_hourly['temp_celsius'] = df_hourly['temp_celsius'] - 273.15
df_hourly['sea_surface_temp_c'] = df_hourly['sea_surface_temp_c'] - 273.15


print("\nData per jam berhasil dimuat:")
print(df_hourly.head())
print(f"\nTotal baris data per jam: {len(df_hourly)}")

In [ ]:
# Validasi data dasar
print("Melakukan validasi data...")
print(f"Jumlah baris sebelum filter: {len(df_hourly)}")

# Cek missing values
missing_counts = df_hourly.isnull().sum()
print(f"Missing values per kolom:\n{missing_counts}")

# Filter data yang valid (hapus baris dengan NaN jika ada)
df_hourly = df_hourly.dropna()

print(f"Jumlah baris setelah filter: {len(df_hourly)}")

# Cek range nilai yang masuk akal
print("Statistik deskriptif:")
print(df_hourly.describe())

# Pastikan data dalam range fisik yang masuk akal
# Suhu: -50°C sampai 60°C
df_hourly = df_hourly[
    (df_hourly['temp_celsius'] >= -50) & (df_hourly['temp_celsius'] <= 60) &
    (df_hourly['sea_surface_temp_c'] >= -2) & (df_hourly['sea_surface_temp_c'] <= 35) &
    (df_hourly['precipitation_mm'] >= 0) & (df_hourly['precipitation_mm'] <= 500) &  # mm/jam
    (df_hourly['wave_height_meters'] >= 0) & (df_hourly['wave_height_meters'] <= 20)
]

print(f"Jumlah baris setelah validasi range: {len(df_hourly)}")
print("Validasi selesai.")

### Langkah 3.3: Agregasi Data (Membuat Rata-rata)

Data Anda sekarang per jam. Sesuai tujuan Anda, kita perlu membuat rata-rata Harian, Mingguan, dan Bulanan. Kita menggunakan fungsi resample() dari Pandas.

In [ ]:
# Pastikan indeks adalah DatetimeIndex (ini penting untuk resample)
df_hourly.index = pd.to_datetime(df_hourly.index)

print("Membuat agregat Harian ('D')...")
df_daily = df_hourly.resample('D').mean()

print("Membuat agregat Mingguan ('W')...")
df_weekly = df_hourly.resample('W').mean()

print("Membuat agregat Bulanan ('M')...")
df_monthly = df_hourly.resample('M').mean()

print("\nContoh data rata-rata bulanan:")
print(df_monthly.head())

### Langkah 3.4: Membuat Tabel Klimatologi (Lookup)

Ini adalah bagian terpenting untuk aplikasi Anda ("melihat data cuaca rata2 pada hari/minggu/bulan"). Kita akan membuat tabel lookup yang berisi rata-rata 10 tahun untuk setiap hari dalam setahun (1 Jan, 2 Jan, dst.) dan setiap bulan (Jan, Feb, dst.).

In [ ]:
print("Membuat tabel lookup klimatologi (rata-rata 10 tahun)...")

# 1. Rata-rata per HARI (1 s/d 366)
# Mengelompokkan berdasarkan "hari dalam setahun"
df_climatology_day_of_year = df_hourly.groupby(df_hourly.index.dayofyear).mean()
df_climatology_day_of_year.index.name = 'day_of_year' # (1 = 1 Jan, 366 = 31 Des)

# 2. Rata-rata per BULAN (1 s/d 12)
# Mengelompokkan berdasarkan "bulan"
df_climatology_month = df_hourly.groupby(df_hourly.index.month).mean()
df_climatology_month.index.name = 'month' # (1 = Jan, 12 = Des)

print("\nContoh tabel Klimatologi per Bulan (rata-rata 10 tahun):")
print(df_climatology_month.head())

## Fase 4: Pembuatan Database SQLite Final

Kita sekarang memiliki 5 tabel (DataFrame) yang siap disimpan ke dalam satu file database.

### Langkah 4.1: Optimasi Indeks Tanggal (Opsional tapi Direkomendasikan)

Untuk kueri yang lebih cepat di Android, sebaiknya simpan tanggal sebagai string yang dapat diindeks (YYYY-MM-DD) atau sebagai Unix timestamp (Integer) daripada objek Datetime lengkap.

In [ ]:
# Mengonversi indeks Datetime menjadi string YYYY-MM-DD untuk kompatibilitas
df_daily.index = df_daily.index.strftime('%Y-%m-%d')
df_weekly.index = df_weekly.index.strftime('%Y-%m-%d')
df_monthly.index = df_monthly.index.strftime('%Y-%m')

# Menetapkan label indeks yang jelas untuk SQLite
df_daily.index.name = 'date'
df_weekly.index.name = 'week_start_date'
df_monthly.index.name = 'month'

print("Indeks telah dikonversi ke string untuk penyimpanan SQLite.")

### Langkah 4.2: Menyimpan DataFrame ke SQLite

Kita akan menggunakan pustaka sqlite3 bawaan Python dan metode .to_sql() dari Pandas.

In [ ]:
import sqlite3

db_filename = 'aceh_weather_data.sqlite'

print(f"Membuat koneksi ke database: {db_filename}")
con = sqlite3.connect(db_filename)

# Menyimpan setiap DataFrame sebagai tabel terpisah
print("Menyimpan tabel: daily_timeline...")
df_daily.to_sql('daily_timeline', con, if_exists='replace', index=True)

print("Menyimpan tabel: weekly_timeline...")
df_weekly.to_sql('weekly_timeline', con, if_exists='replace', index=True)

print("Menyimpan tabel: monthly_timeline...")
df_monthly.to_sql('monthly_timeline', con, if_exists='replace', index=True)

print("Menyimpan tabel: climatology_by_day...")
df_climatology_day_of_year.to_sql('climatology_by_day', con, if_exists='replace', index=True)

print("Menyimpan tabel: climatology_by_month...")
df_climatology_month.to_sql('climatology_by_month', con, if_exists='replace', index=True)

# Menutup koneksi
con.close()

print(f"\n*** SELURUH PROSES SELESAI. ***")
print(f"Database '{db_filename}' telah berhasil dibuat di lingkungan Colab.")

### Langkah 4.3: Mengunduh File Database Final dari Colab

Setelah sel di atas selesai, jalankan sel terakhir ini. Sel ini akan memicu unduhan browser untuk file aceh_weather_data.sqlite dari server Colab ke komputer lokal Anda.

In [ ]:
from google.colab import files

print(f"Mempersiapkan unduhan untuk: {db_filename}")
files.download(db_filename)

# Zip dan download folder data_iklim
import shutil
shutil.make_archive('data_iklim', 'zip', 'data_iklim')
print("Mempersiapkan unduhan untuk: data_iklim.zip")
files.download('data_iklim.zip')